## Video Surveillance: Building an Object Tracker
#### Using your laptop/desktop webcam, create a video of a moving object having a distant colour with respect to the background.
#### a) Implement a mean-shift tracking algorithm for this particular object.
#### b) Try to see the output of the algorithm on rotating the object and changing the distance of the object from the camera.

## importing libraries/library

In [1]:
import numpy as np
import pandas as pd

In [3]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2

In [3]:
# reading the data from an already existing video
cap = cv2.VideoCapture('video.mp4')
cap.set(cv2.CAP_PROP_POS_FRAMES, 500)

# reading the first frame to get detials about the object
success,frame = cap.read()


# getting shape of the whole frame in the video
frame_h,frame_w = frame.shape[:2]
size = (frame_h,frame_w)

# setting values for position of the object 
w =frame_w//4
h =frame_h//4
x =600
y =320
track_window = (x,y,w,h)

# cropping the object from the whole frame(or getting region of intrest(roi))
roi = frame[y:y+h, x-w:x]

# converting color system BGR to HSV(for better understanding of shadows and brightness or contrast)
hsv_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
mask = None

# calculating the histogram values of object to know difference btw bakground and object
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0,180])
# and normalizing the histogram values to a range of 0-255
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# the criteria required for mean-shift algorithm to work
term_criteria =(cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS,10,1)

# creating a object to write a video or save a new video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
result = cv2.VideoWriter('phone.mp4',fourcc,10,(int(cap.get(3)),int(cap.get(4))))

# reading the next frame in video 
true,frame = cap.read()
while true:
    # converting color system BGR to HSV
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    # masking the background in the frame
    back_project = cv2.calcBackProject([hsv], [0],roi_hist, [0,180],1)
    
    # applying the mean-shift algorithm
    num_iters,track_window = cv2.meanShift(back_project,track_window,term_criteria)
    
    # the next position of object after it has moved or changed position 
    x,y,w,h = track_window
    
    # drawing rectangle on new position of object
    cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
    
    # showing the masked background
    cv2.imshow('back-projection',back_project)
    
    # showing the original frame with object detected
    cv2.imshow('meanshift', frame)
    
    # writing the frame in the video 
    result.write(frame)

    # time-gap for image to get removed from output screen
    k = cv2.waitKey(50)
    if k == 27:
        break
    if k == ord('p'):
        cv2.waitKey(-1)

    true,frame = cap.read()
    
cv2.destroyAllWindows()